In [1]:
!Is-Iha kaggle.json
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.10

/bin/bash: Is-Iha: command not found
Uninstalling kaggle-1.5.10:
  Successfully uninstalled kaggle-1.5.10
     |████████████████████████████████| 1.5MB 16.9MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 59 kB 7.2 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.10-py3-none-any.whl size=73268 sha256=556a2217304c6f0a1076ea0ece535c885322450934c6723e24b51b4b1b14a26f
  Stored in directory: /root/.cache/pip/wheels/ea/c5/fe/7e7fb5b3d1f150fac96188949b3d83d375a4c9df16ba557e52
Successfully built kaggle


In [2]:
#json파일 확인
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle -v

Kaggle API 1.5.10


In [3]:
!kaggle competitions download -c nlp-getting-started

  0% 0.00/593k [00:00<?, ?B/s]
100% 593k/593k [00:00<00:00, 122MB/s]


In [4]:
!unzip nlp-getting-started

Archive:  nlp-getting-started.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [139]:
import pandas as pd
import numpy as np
import torch
import random
import torch.nn as nn

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [7]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
submit=pd.read_csv("sample_submission.csv")

In [8]:
train

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


keyword, location은 삭제


In [9]:
train=train.drop(labels="keyword",axis=1)
train=train.drop(labels='location',axis=1)

test=test.drop("keyword",axis=1)
test=test.drop("location",axis=1)




---



In [ ]:
train.to_csv("train_data.csv",index=False)
test.to_csv("test_data.csv",index=False)

train_data=pd.read_csv("train_data.csv")
test_data=pd.read_csv("test_data.csv")

In [15]:
test

,id,text
0,0,Just happened a terrible car crash
1,2,"Heard about #earthquake is different cities, s..."
2,3,"there is a forest fire at spot pond, geese are..."
3,9,Apocalypse lighting. #Spokane #wildfires
4,11,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...
3258,10861,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,Storm in RI worse than last hurricane. My city...
3260,10868,Green Line derailment in Chicago http://t.co/U...
3261,10874,MEG issues Hazardous Weather Outlook (HWO) htt...


In [10]:
from sklearn.model_selection import train_test_split

train_data,val_data=train_test_split(train,test_size=0.1,shuffle=True,random_state=71)
len(train_data),len(val_data),len(test)

(6851, 762, 3263)

In [11]:
train_data

,id,text,target
7183,10293,@junsuisengen changing my weapon!,0
5026,7167,@SophieWisey I couldn't. #mudslide,1
1190,1715,Leicester_Merc : ICYMI - #Ashes 2015: Australi...,0
1196,1722,'i'm a Gemini' *children screaming buildings b...,1
1386,1999,Ted Cruz fires back at Jeb &amp; Bush: ÛÏWe l...,0
...,...,...,...
1576,2277,I hope they fall off a cliff.,0
5175,7381,Meek Mill Begging Nicki Minaj To Let Him Oblit...,0
3083,4425,She says that she'd love to come help but\nThe...,0
4013,5699,Who is bringing the tornadoes and floods. Who ...,0


#torctext

버전이 바뀌면서 기존의 방법들은 실행안됨 

0.8.1 --> 0.9.0

In [12]:
!pip install torchtext

In [13]:
import torchtext
from torchtext import data

In [39]:
#tokenizer로 Spacy를 사용할 것
#이외에도 NLTK, 영어는 띄어쓰기만으로도 tokenizer가 잘 됨 (한국어는 아님)
import spacy

spacy_en=spacy.load('en')

In [15]:

#필드 정의

TEXT=data.Field(sequential=True,use_vocab=True,tokenize="spacy",
                lower=True,batch_first=True,fix_length=1000)

LABEL=data.Field(sequential=False,use_vocab=False,
                lower=False,batch_first=True,is_target=True,
                 #preprocessing= lambda x: int (x)
                 )

AttributeError: ignored

In [16]:
from torchtext.data import TabularDataset


train_data, test_data = TabularDataset.splits(
        path='.', train='train_data.csv', test='test_data.csv', format='csv',
        fields=[('text', TEXT), ('label', LABEL)], skip_header=True)

ImportError: ignored

In [ ]:
print(vars(train_data[0]))
print(vars(test_data[0]))

{'text': ['our', 'deeds', 'are', 'the', 'reason', 'of', 'this', '#', 'earthquake', 'may', 'allah', 'forgive', 'us', 'all'], 'label': '1'}
{'text': ['just', 'happened', 'a', 'terrible', 'car', 'crash']}


In [17]:
class ClassifyDataset(torchtext.data.Dataset):
    def __init__(self, df, fields, train=True, **kwargs):
        examples=[]
        for i, row in df.iterrows():
            examples.append(torchtext.data.Example.fromlist([row.text, row.target if train else None],fields))
        super().__init__(examples, fields, **kwargs)
        
    @staticmethod
    def sort_key(x):
        return len(x.text)
    
    @classmethod
    def splits(cls, fields, train_df=None, val_df=None, test_df=None, **kwargs):
        train_data, val_data, test_data=(None, None, None)
        
        if train_df is not None:
            train_data=cls(train_df.copy(), fields, **kwargs)
            
        if val_df is not None:
            val_data=cls(val_df.copy(), fields, **kwargs)
            
        if test_df is not None:
            test_data=cls(test_df.copy(), fields, train=False, **kwargs)
            
        return tuple(d for d in (train_data, val_data, test_data) if d is not None)

AttributeError: ignored

In [18]:
Text=torchtext.data.Field(spacy_en, include_lengths=True)
Label=torchtext.data.LabelField(dtype=torch.float)

fields= [('text', Text),('label', Label)]

train_ds,val_ds, test_ds= ClassifyDataset.splits(fields, train_df=train_data, val_df=val_data ,test_df=test)



AttributeError: ignored

In [19]:
from torchtext.data import Iterator

batch_size=10

train_loader=Iterator(dataset=train_ds,batch_size=batch_size,device="cuda")
val_loader=Iterator(dataset=val_ds,batch_size=batch_size,device="cuda")
test_loader=Iterator(dataset=test_ds,batch_size=batch_size,device="cuda")

ImportError: ignored



---



##torchtext 0.9.0 version

https://pytorch.org/text/0.9.0/data_functional.html

In [40]:
#tokenizer=spacy
from torchtext.vocab import Vectors, Vocab,GloVe
from collections import Counter
from torchtext.data.utils import get_tokenizer

tokenizer = get_tokenizer("spacy")

glove=GloVe(name='6B',dim=300)
counter=Counter()

for i in train_data.index:
  counter.update(tokenizer(train_data['text'][i]))


In [41]:
counter

Counter({'@junsuisengen': 1,
         'changing': 2,
         'my': 489,
         'weapon': 33,
         '!': 1046,
         '@SophieWisey': 1,
         'I': 1416,
         'could': 63,
         "n't": 402,
         '.': 2589,
         '#': 3028,
         'mudslide': 26,
         'Leicester_Merc': 1,
         ':': 1707,
         'ICYMI': 3,
         '-': 1260,
         'Ashes': 10,
         '2015': 66,
         'Australia': 17,
         'collapse': 51,
         'at': 444,
         'Trent': 11,
         'Bridge': 13,
         'how': 101,
         'Twitter': 20,
         'rea\x89Û': 8,
         '_': 377,
         'http://t.co/HqeWMREysO': 1,
         ')': 288,
         'http://t.co/y4y8fclJED': 1,
         "'": 1166,
         'i': 172,
         "'m": 212,
         'a': 1683,
         'Gemini': 1,
         '*': 156,
         'children': 18,
         'screaming': 21,
         'buildings': 80,
         'burning': 79,
         'police': 71,
         'sirens': 13,
         'in': 1612,
       

In [42]:
vocabulary=Vocab(counter, max_size=20000, min_freq=2, vectors=glove, specials=['<pad>', '<unk>'])

print('Embedding vocab size: ', vocabulary.vectors.size(0))

Embedding vocab size:  7612


In [60]:
vocabulary.stoi

#7612*300

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7f889de18590>>,
            {'<pad>': 0,
             '<unk>': 1,
             '#': 2,
             '?': 3,
             '.': 4,
             'the': 5,
             ':': 6,
             'a': 7,
             'to': 8,
             'in': 9,
             'of': 10,
             'I': 11,
             '-': 12,
             'and': 13,
             "'": 14,
             '!': 15,
             '...': 16,
             'is': 17,
             'for': 18,
             'on': 19,
             "'s": 20,
             'you': 21,
             'it': 22,
             'The': 23,
             ' ': 24,
             'that': 25,
             'my': 26,
             '\n': 27,
             'with': 28,
             'at': 29,
             'by': 30,
             ';': 31,
             "n't": 32,
             '_': 33,
             'are': 34,
             'was': 35,
             'be': 36,
             'have': 37,
             '&': 38,


In [132]:
#예시
examples = ['chip', 'baby', 'Beautiful']
vec = torchtext.vocab.GloVe(name='6B', dim=50)
ret = vec.get_vecs_by_tokens(examples, lower_case_backup=True)

ret

tensor([[-0.7710, -1.1697,  1.5195,  0.8371,  0.7419, -0.2185, -0.7212, -0.9400,
         -0.0113,  0.5485,  0.4040, -0.1846, -0.4630,  0.2620, -0.6464,  0.3599,
         -0.8610, -0.3869, -0.0271, -1.0254,  0.3280, -0.7500, -0.6859, -0.6912,
          0.3429, -0.6660, -0.2910, -0.6104,  0.3322, -0.4252,  2.4573, -0.8748,
          0.4891,  1.2888,  0.5780, -0.5509, -0.2263,  0.8127,  0.7048, -0.5498,
          0.3620, -0.2171, -0.2991,  0.2917,  1.2260,  0.2446,  1.2133, -0.0967,
          0.0474,  0.1971],
        [ 0.5494,  0.2299, -0.0357, -0.9143,  0.7044,  1.3736, -0.9937, -0.5034,
          0.5793,  0.3481,  0.2385,  0.5444,  0.3432,  0.5741,  1.3732,  0.4636,
         -0.7288,  0.2887,  0.1001, -0.2302, -0.1289,  0.7033,  0.3961,  0.2605,
          0.2697, -1.3036, -0.9377,  0.2705,  0.6070, -0.6689,  1.9709,  0.6796,
         -0.6944,  1.0380,  0.5136,  0.2302,  0.3646, -0.3090,  1.1395, -1.1466,
         -0.7889,  0.0544, -0.0691, -0.2439,  1.4049,  0.0919,  0.2313, -1.3028,


In [93]:
y_train=np.array(train_data['target'])
y_val=np.array(val_data['target'])


y_train=torch.LongTensor(y_train)
y_val=torch.LongTensor(y_val)

In [132]:
#vocabulary.vectors.size(0)
len(vocabulary)

7612

##dataset,dataloader

버전 0.8.1에서 소스코드 가져다 쓰면 될거 같긴 하다..

https://pytorch.org/text/0.8.1/data.html

dataset 만들기-> 딕셔너리 사용

In [120]:
train_data_set={}
train_data_set_list=[]
for i in train_data.index:
  train_data_set={'text': tokenizer(train_data['text'][i]),'label': train_data['target'][i]}
  train_data_set_list.append(train_data_set)

In [121]:
val_data_set={}
val_data_set_list=[]
for i in val_data.index:
  val_data_set={'text': tokenizer(val_data['text'][i]),'label': val_data['target'][i]}
  val_data_set_list.append(val_data_set)

In [122]:
train_data_set_list[0]

{'label': 0, 'text': ['@junsuisengen', 'changing', 'my', 'weapon', '!']}

In [127]:
batch_size=10

In [128]:
train_loader = torch.utils.data.DataLoader(dataset=train_data_set_list,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True,
                                          num_workers=1)

val_loader = torch.utils.data.DataLoader(dataset=val_data_set_list,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True,
                                         num_workers=1)

In [129]:
len(train_loader),len(val_loader)

(685, 76)

#모델 - 일단 알고있는 모델인 CNN모델로 해보기

In [143]:
# Hyperparameters
num_epochs = 25
learning_rate = 0.001

INPUT_DIM = len(vocabulary)
EMBEDDING_DIM = 300
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.2
PAD_IDX = vocabulary.stoi['<pad>'] # padding


In [145]:
class LSTM_net(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc1 = nn.Linear(hidden_dim * 2, hidden_dim)
        
        self.fc2 = nn.Linear(hidden_dim, 1)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        # text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        # embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        #unpack sequence
        # output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        # output = [sent len, batch size, hid dim * num directions]
        # output over padding tokens are zero tensors
        
        # hidden = [num layers * num directions, batch size, hid dim]
        # cell = [num layers * num directions, batch size, hid dim]
        
        # concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        # and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        output = self.fc1(hidden)
        output = self.dropout(self.fc2(output))
                
        #hidden = [batch size, hid dim * num directions]
            
        return output

In [146]:
model = LSTM_net(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [147]:
pretrained_embeddings = vocabulary.vectors

print(pretrained_embeddings.shape)
model.embedding.weight.data.copy_(pretrained_embeddings)

torch.Size([7612, 300])


tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0690, -0.0789,  0.5624,  ..., -0.1015,  0.4912,  0.5452],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.1434, -0.4603,  0.0370,  ...,  0.0772, -0.0917,  0.6458],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [148]:
#  to initiaise padded to zeros
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0690, -0.0789,  0.5624,  ..., -0.1015,  0.4912,  0.5452],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.1434, -0.4603,  0.0370,  ...,  0.0772, -0.0917,  0.6458],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])


In [149]:
model.to(device) #CNN to GPU


# Loss and optimizer
criterion = nn.BCEWithLogitsLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)